In [0]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image

In [0]:
(x_train, x_test), (y_train, y_test) = mnist.load_data()
train_x = x_train.reshape(60000, 784) / 255
val_x  = y_train.reshape(10000, 784) / 255

In [0]:
autoencoder = Sequential(name="mnist_anomaly")
autoencoder.add(Dense(512,  activation='elu', input_shape=(784,)))
autoencoder.add(Dense(128,  activation='elu'))
autoencoder.add(Dense(10,    activation='linear', name="bottleneck"))
autoencoder.add(Dense(128,  activation='elu'))
autoencoder.add(Dense(512,  activation='elu'))
autoencoder.add(Dense(784,  activation='sigmoid'))
autoencoder.compile(loss='mean_squared_error', optimizer = Adam())

In [54]:
trained_model = autoencoder.fit(train_x, train_x, batch_size=1024, epochs=10, verbose=1, validation_data=(val_x, val_x))

Epoch 1/10
59/59 [==============================] - 1s 14ms/step - loss: 0.0752 - val_loss: 0.0485
Epoch 2/10
59/59 [==============================] - 1s 12ms/step - loss: 0.0431 - val_loss: 0.0377
Epoch 3/10
59/59 [==============================] - 1s 12ms/step - loss: 0.0345 - val_loss: 0.0312
Epoch 4/10
59/59 [==============================] - 1s 12ms/step - loss: 0.0303 - val_loss: 0.0282
Epoch 5/10
59/59 [==============================] - 1s 13ms/step - loss: 0.0275 - val_loss: 0.0256
Epoch 6/10
59/59 [==============================] - 1s 12ms/step - loss: 0.0250 - val_loss: 0.0235
Epoch 7/10
59/59 [==============================] - 1s 12ms/step - loss: 0.0233 - val_loss: 0.0222
Epoch 8/10
59/59 [==============================] - 1s 12ms/step - loss: 0.0222 - val_loss: 0.0212
Epoch 9/10
59/59 [==============================] - 1s 12ms/step - loss: 0.0212 - val_loss: 0.0204
Epoch 10/10
59/59 [==============================] - 1s 12ms/step - loss: 0.0204 - val_loss: 0.0196


In [0]:
#encoder
encoder = Model(autoencoder.input, autoencoder.get_layer("bottleneck").output)
encoded_data = encoder.predict(train_x) #bottleneck representation

decoded_output = autoencoder.predict(train_x) #reconstruction loss

encoding_dims = 10 #must match bottleneck hidden units

#return decoder
encoded_input = Input(shape=(encoding_dims, ))
decoder = autoencoder.layers[-3](encoded_input)
decoder = autoencoder.layers[-2](decoder)
decoder = autoencoder.layers[-1](decoder)
decoder = Model(encoded_input, decoder)

In [57]:
# if the layout.jpg is not one of the MNIST dataset that the model was trained on, the error will be very high.
img = image.load_img("layout.JPG", target_size=(28, 28), color_mode = "grayscale")
input_img = image.img_to_array(img)
inputs = input_img.reshape(1, 784)

target_data = autoencoder.predict(inputs)

dist = np.linalg.norm(inputs - target_data, axis=-1)
print(dist)

[6799.5938]


In [58]:
inputs = train_x[0].reshape(1, 784)
target_data = autoencoder.predict(inputs)
dist = np.linalg.norm(inputs - target_data, axis=-1)
print(dist)

[5.65472872]
